In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def join_coordinates(df):
    coordinates = df['centroid'].str.extract(r'Point \((\d+\.?\d*) (\d+\.?\d*)\)', expand=True)
    coordinates.columns = ['x', 'y']
    coordinates['x'] = coordinates['x'].astype('float64')
    coordinates['y'] = coordinates['y'].astype('float64')
    return df.join(coordinates)

In [ ]:
df15 = join_coordinates(df15)
df16 = join_coordinates(df16)
df17 = join_coordinates(df17)
df18 = join_coordinates(df18)
df19 = join_coordinates(df19)

In [ ]:
def find_prefix_by_matching_str(prefixes, string):
    for prefix in prefixes:
        if string.startswith(prefix):
            return prefix
    return None

312

In [ ]:
!ls /content/gdrive/MyDrive/cultures/
!pwd
!mkdir -p cultures
!ls

In [ ]:
!unzip /content/gdrive/MyDrive/cultures/Dossier-PAC-2020_notice_cultures-precisions-pages-1-6.zip -d /content/cultures/2020/
!unzip /content/gdrive/MyDrive/cultures/Dossier-PAC-2019_notice_cultures-precisions-pages-1-6.zip -d /content/cultures/2019/
!unzip /content/gdrive/MyDrive/cultures/Dossier-PAC-2018_notice_cultures-precisions-pages-1-6.zip -d /content/cultures/2018/
!unzip /content/gdrive/MyDrive/cultures/Dossier-PAC-2017_notice_cultures-precisions-pages-1-6.zip -d /content/cultures/2017/
!unzip /content/gdrive/MyDrive/cultures/Dossier-PAC-2016_notice_cultures-precisions-pages-1-6.zip -d /content/cultures/2016/
!unzip /content/gdrive/MyDrive/cultures/Dossier-PAC-2015_notice_cultures-precisions-pages-1-6.zip -d /content/cultures/2015/

Archive:  /content/gdrive/MyDrive/cultures/Dossier-PAC-2020_notice_cultures-precisions-pages-1-6.zip
replace /content/cultures/2020/rawText.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/gdrive/MyDrive/cultures/Dossier-PAC-2019_notice_cultures-precisions-pages-1-6.zip
replace /content/cultures/2019/rawText.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/gdrive/MyDrive/cultures/Dossier-PAC-2018_notice_cultures-precisions-pages-1-6.zip
replace /content/cultures/2018/rawText.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/gdrive/MyDrive/cultures/Dossier-PAC-2017_notice_cultures-precisions-pages-1-6.zip
replace /content/cultures/2017/rawText.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/gdrive/MyDrive/cultures/Dossier-PAC-2016_notice_cultures-precisions-pages-1-6.zip
replace /content/cultures/2016/rawText.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/gdrive/MyDrive/cultures/Dossier-PAC-2015_notice_cultures-precision

In [ ]:
import re
def parse_cultures_tables(dir_with_tables):
    cool_column_prefixes = ['Libellé de la culture', 'Code de la culture']
    cultures = pd.DataFrame(columns=cool_column_prefixes)
    for filename in os.listdir(dir_with_tables):
        if re.match(r'table-\d+\.csv', filename): 
            table = pd.read_csv(f'{dir_with_tables}/{filename}')
            # collect needed columns
            cols = [column for column in table.columns 
                    if any([column.startswith(cool_column_prefix) for cool_column_prefix in cool_column_prefixes])]
            if len(cols) == 0:
                continue
            table = table[cols]
            # strip columns and values
            table.columns = [col.strip() for col in table.columns]
            for column in table.columns:
                try:
                    table.loc[:, column] = table[column].str.strip().copy()
                except Exception as e:
                    print(f'Error processing file({dir_with_tables}/{filename}), column({column})', e)
            # some results contains actual several table which are concatenated horizontaly
            # so we are going to sprit it
            postfixes = set([col[-3:] for col in table.columns])
            grouped_cols = [[col for col in table.columns if col.endswith(postfix)] for postfix in postfixes]
            for col_group in grouped_cols:
                subtable = table[col_group].copy()
                # normalize column names
                subtable.columns = [find_prefix_by_matching_str(cool_column_prefixes, col) for col in subtable.columns]
                subtable = subtable.dropna(subset=[cool_column_prefixes[1]])
                cultures = cultures.append(subtable, ignore_index=True)
    cultures.columns = ['description', 'code']
    return cultures

In [ ]:
cultures20 = parse_cultures_tables('/content/cultures/2020')
cultures19 = parse_cultures_tables('/content/cultures/2019')
cultures18 = parse_cultures_tables('/content/cultures/2018')
cultures17 = parse_cultures_tables('/content/cultures/2017')
cultures16 = parse_cultures_tables('/content/cultures/2016')
cultures15 = parse_cultures_tables('/content/cultures/2015')
cultures_list = [cultures20, cultures19, cultures18, cultures17, cultures16, cultures15]

Error processing file(/content/cultures/2017/table-8.csv), column(Libellé de la culture) Can only use .str accessor with string values!
Error processing file(/content/cultures/2016/table-9.csv), column(Libellé de la culture) Can only use .str accessor with string values!


In [ ]:
for cultures in cultures_list:
    print(f'Total cultures vs unique cultures vs unique descriptions: {cultures.shape[0]} vs {cultures.iloc[:, 1].nunique()} vs {cultures.iloc[:, 0].nunique()}')

Total cultures vs unique cultures vs unique descriptions: 278 vs 278 vs 278
Total cultures vs unique cultures vs unique descriptions: 278 vs 278 vs 278
Total cultures vs unique cultures vs unique descriptions: 323 vs 323 vs 322
Total cultures vs unique cultures vs unique descriptions: 322 vs 322 vs 321
Total cultures vs unique cultures vs unique descriptions: 304 vs 304 vs 303
Total cultures vs unique cultures vs unique descriptions: 290 vs 290 vs 290


In [ ]:
cultures20['year'] = 2020
cultures19['year'] = 2019
cultures18['year'] = 2018
cultures17['year'] = 2017
cultures16['year'] = 2016
cultures15['year'] = 2015

In [ ]:
cultures = pd.concat(cultures_list, ignore_index=True)

In [ ]:
cultures.head()

,description,code,year
0,Autre céréale de genre Fagopyrum,CGF,2020
1,Autre céréale ou pseudo-céréale d'un autre genre*,CAG,2020
2,Mélange de céréales ou pseudo-céréales pures o...,MCR,2020
3,Mélisse,MLI,2020
4,Menthe,MTH,2020


In [ ]:
cultures['code'].nunique()

350

In [ ]:
print(f'Unique cultures vs unique descriptions: {cultures.iloc[:, 1].nunique()} vs {cultures.iloc[:, 0].nunique()}')

Unique cultures vs unique descriptions: 350 vs 415


In [ ]:
cultures[cultures['description'].isna()]

,description,code,year
714,NaN,ROS,2018
1089,NaN,LIF,2017
1201,NaN,LIF,2016


In [ ]:
cultures[((cultures['code'] == 'LIF') | (cultures['code'] == 'ROS')) & (~cultures['description'].isna())]

,description,code,year
154,Lin fibres,LIF,2020
158,Roselière,ROS,2020
441,Roselière,ROS,2019
466,Lin fibres,LIF,2019
775,Lin fibres,LIF,2018
960,Roselière,ROS,2017
1458,Roselière,ROS,2016
1505,Lin fibres,LIF,2015
1541,Roselière,ROS,2015


In [ ]:
cultures[cultures['code'] == 'LIF'] = 'Lin fibres'
cultures[cultures['code'] == 'ROS'] = 'Roselière'
cultures[cultures['description'].isna()]

,description,code,year


In [ ]:
cultures.to_csv('gdrive/MyDrive/cultures/cultures.csv', index=False, quoting=1)